In [12]:
import pandas as pd
import matplotlib.pyplot as plt

# 한글화
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False

In [13]:
# 1. 데이터 불러오기
df = pd.read_csv('한국_기업문화_HR_데이터셋_샘플.csv')

# 2. 데이터 확인
print(df.head())

   Unnamed: 0  Age 이직여부           출장빈도  일일성과지표                      부서  집까지거리  \
0        1041   28   No  Travel_Rarely     866                   Sales      5   
1         184   53   No  Travel_Rarely    1084  Research & Development     13   
2        1222   24  Yes  Travel_Rarely     240         Human Resources     22   
3          67   45   No  Travel_Rarely    1339  Research & Development      7   
4         220   36   No  Travel_Rarely    1396  Research & Development      5   

   학력수준             전공분야  EmployeeCount  ...  대인관계만족도  StandardHours 스톡옵션등급  \
0     3          Medical              1  ...        4             80      0   
1     2          Medical              1  ...        3             80      2   
2     1  Human Resources              1  ...        3             80      1   
3     3    Life Sciences              1  ...        3             80      1   
4     2    Life Sciences              1  ...        4             80      0   

   총경력  연간교육횟수  워라밸 현회사근속년수  현직무근속년수 최

In [14]:
# 3단계. 칼럼명 한글에서 영어로 변경 
df.rename(columns={
    '이직여부': 'Attrition',
    '야근여부': 'OverTime',
    '업무만족도': 'JobSatisfaction',
    '워라밸': 'WorkLifeBalance',
    '집까지거리': 'DistanceFromHome',
    '현회사근속년수': 'YearsAtCompany',
    '출장빈도': 'BusinessTravel',
}, inplace=True)

In [15]:
# 4단계. 칼럼 선별 6개 (이직율과 가장 높은 상관관계 6개 선정)

df_selected = df[['Attrition', #  예측 대상 변수 (이직 여부: Yes/No)
                  'OverTime', #  야근 여부 –> 스트레스, 이직률과 매우 높은 상관
                  'JobSatisfaction', #  업무 만족도 –> 낮을수록 이직 가능성↑
                  'WorkLifeBalance',  #  워라밸 만족도 –> 조직문화·불만족 요인
                  'DistanceFromHome',  #  집과 직장 간 거리 –> 통근 피로도는 주요 이직 사유
                  'YearsAtCompany', #  현 회사 근속 연수 –> 짧을수록 이직률 높음
                  'BusinessTravel']]   #  출장 빈도 –> 잦은 출장은 소진과 이직에 영향

# 확인
print(df_selected.head())


  Attrition OverTime  JobSatisfaction  WorkLifeBalance  DistanceFromHome  \
0        No       No                1                3                 5   
1        No       No                1                3                13   
2       Yes       No                3                3                22   
3        No       No                1                3                 7   
4        No       No                2                4                 5   

   YearsAtCompany BusinessTravel  
0               5  Travel_Rarely  
1               4  Travel_Rarely  
2               1  Travel_Rarely  
3               1  Travel_Rarely  
4              13  Travel_Rarely  


In [16]:
# 5단계 머신러닝을 위한 데이터 변환 (범주형 변수 -> 숫자형 변수 변환)
from sklearn.preprocessing import LabelEncoder

# 야근빈도와 이직여부는 이진범주라 LabelEncoding
df_selected['Attrition'] = df_selected['Attrition'].map({'Yes': 1, 'No': 0})
df_selected['OverTime'] = df_selected['OverTime'].map({'Yes': 1, 'No': 0})

# 출장은 One-Hot 정수형 변환
df_selected = pd.get_dummies(df_selected, columns=['BusinessTravel'], drop_first=True, dtype=int)

# 결과 확인
print(df_selected.head())

   Attrition  OverTime  JobSatisfaction  WorkLifeBalance  DistanceFromHome  \
0          0         0                1                3                 5   
1          0         0                1                3                13   
2          1         0                3                3                22   
3          0         0                1                3                 7   
4          0         0                2                4                 5   

   YearsAtCompany  BusinessTravel_Travel_Frequently  \
0               5                                 0   
1               4                                 0   
2               1                                 0   
3               1                                 0   
4              13                                 0   

   BusinessTravel_Travel_Rarely  
0                             1  
1                             1  
2                             1  
3                             1  
4                             

C:\Users\chlwo\AppData\Local\Temp\ipykernel_94440\3773233403.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['Attrition'] = df_selected['Attrition'].map({'Yes': 1, 'No': 0})
C:\Users\chlwo\AppData\Local\Temp\ipykernel_94440\3773233403.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['OverTime'] = df_selected['OverTime'].map({'Yes': 1, 'No': 0})


In [ ]:
# 6단계 데이터 전처리(결측치 제거)
print(df_selected.isnull().sum())
# 이미 결측치가 없으니 이상치만 보자.

Attrition                           0
OverTime                            0
JobSatisfaction                     0
WorkLifeBalance                     0
DistanceFromHome                    0
YearsAtCompany                      0
BusinessTravel_Travel_Frequently    0
BusinessTravel_Travel_Rarely        0
dtype: int64


In [ ]:
# 7단계 데이터 전처리(이상치 처리)

# IQR 방식으로 이상치 확인 출처 : https://velog.io/@jihyunko/%EC%9D%B4%EC%83%81%EC%B9%98-%EC%89%BD%EA%B2%8C-%EC%B2%98%EB%A6%AC%ED%95%98%EA%B8%B0

def findOutliers(x, column):
    q1 = x[column].quantile(0.25)
    q3 = x[column].quantile(0.75)
    iqr = q3 - q1
    y = x[(x[column] > (q3 + 1.5 * iqr)) | (x[column] < (q1 - 1.5 * iqr))]
    return y  # 이상치만 반환

# 예: DistanceFromHome 컬럼의 이상치 확인
outliers_df = findOutliers(df_selected, 'DistanceFromHome')
print("이상치 개수:", len(outliers_df))
print(outliers_df[['DistanceFromHome']])


ImportError: cannot import name 'sns' from 'seaborn' (c:\anaconda\Lib\site-packages\seaborn\__init__.py)